
# EEG Artifact removal using ICA


    ICA Filtering algorithm uses following three criteria for removing artifacts for EEG.

    1. Kurtosis based artifacts - mostly for motion artifacts:
        -  parameter `kur_thr` is used as threshold on kurtosis of ICs. Any IC above `kur_thr`
           is removed. As higher kurtosis of component, more peaky it is.
    
    2. Correlation Based Index (CBI) for eye movement artifacts:
        -  CBI method [1]_ computed the comparaision of power in prefrontal electrodes with frontal eletrodes, in IC.
           A component, that stisfy the criteria, is considered as component capturing eye-blink activity and removed.
        -  For applying CBI method, index of prefrontal (AF - First Layer of electrodes towards frontal lobe)
           and frontal lobe (F - second layer of electrodes) channels needs to be provided.

        -  For case of 14-channels Emotiv Epoc
              - ch_names = ['AF3','F7','F3','FC5','T7','P7','O1','O2','P8','T8','FC6','F4','F8','AF4']
              - Pre-frontal Channels =['AF3','AF4'], 
              - Fronatal Channels = ['F7','F3','F4','F8']
              - then suplied index of channels are as follow;
                   -  `AF_ch_index` =[0,13] 
                   -  `F_ch_index` =[1,2,11,12] 

        - .. [1] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3812628/
    
    3. Correlation of any IC with many EEG channels:
        -  If any IC is correlated `corr_thr` % (80%) of elecctrodes, is considered to be artifactual
        -  Similar like CBI, except, not comparing fronatal and prefrontal but to all


In [ ]:
# Importing libraries/spkit
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import spkit as sp
print('spkit version :', sp.__version__)

## Load and filter EEG Signal



In [ ]:
# Load sample EEG Data ( 16 sec, 128 smapling rate, 14 channel)
# Filter signal (with IIR highpass 0.5Hz)

X, fs, ch_names = sp.data.eeg_sample_14ch()
Xf = sp.filter_X(X.copy(),fs=128.0, band=[0.5], btype='highpass',ftype='SOS')
print(Xf.shape)

## Artifact removal using ICA



In [ ]:
XR = sp.eeg.ICA_filtering(Xf.copy(),verbose=1,kur_thr=2,corr_thr=0.8,winsize=128)




t = np.arange(Xf.shape[0])/fs
plt.figure(figsize=(12,8))
plt.subplot(211)
plt.plot(t,XR+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('XR: Corrected Signal')
plt.subplot(212)
plt.plot(t,(Xf-XR)+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf - XR: Difference (removed signal)')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,5))
plt.plot(t,Xf+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf: 14 channel - EEG Signal (filtered)')
plt.tight_layout()
plt.show()

## On smaller segment



In [ ]:
Xf1 = Xf[128*10:128*14].copy()

t = np.arange(Xf1.shape[0])/fs
plt.figure(figsize=(12,5))
plt.plot(t,Xf1+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf: 14 channel - EEG Signal (filtered)')
plt.tight_layout()
plt.show()


XR1 = sp.eeg.ICA_filtering(Xf1.copy(),verbose=1,kur_thr=2,corr_thr=0.8,winsize=128*2)
print(XR1.shape)

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(t,XR1+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('XR: Corrected Signal')

plt.subplot(122)
plt.plot(t,(Xf1-XR1)+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf - XR: Difference (removed signal)')
plt.tight_layout()
plt.show()